# Import Library

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Set up the driver

In [2]:
driver = webdriver.Chrome()
url = "https://www.nba.com/stats/leaders"
url_season = [
    "?Season=2024-25",
    "?Season=2023-24",
    "?Season=2022-23",
    "?Season=2021-22",
    "?Season=2020-21",
]

data = pd.DataFrame()

# Crawling Data

In [3]:
for i in url_season:
    driver.get(url + i)
    time.sleep(2)  # Allow the page to load

    # Wait for the pagination element to appear
    try:
        pagination = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, '//div[contains(@class, "Pagination_content")]')
            )
        )
        page = int(pagination.find_element(By.XPATH, ".//div[4]").text.split(" ")[-1])
    except Exception as e:
        print(f"Pagination element not found: {e}")
        continue

    # Locate table headers
    headers = driver.find_element(By.XPATH, '//tr[contains(@class, "Crom_headers")]')
    columns = [header.text for header in headers.find_elements(By.TAG_NAME, "th")]
    columns.append("Season")  # Add a season column

    # Temporary DataFrame for the current season
    season_data = pd.DataFrame(columns=columns)

    for _ in range(page):
        # Locate table rows
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, '//tbody[contains(@class, "Crom_body")]')
            )
        )
        rows = table.find_elements(By.XPATH, ".//tr")

        for row in rows:
            cells = row.find_elements(By.XPATH, ".//td")
            row_data = [cell.text for cell in cells]
            if row_data:  # Avoid empty rows
                row_data.append(i.split("=")[-1])  # Add the season
                season_data.loc[len(season_data)] = row_data

        # Click the next page button
        try:
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(
                    (
                        By.XPATH,
                        './/button[@type="button" and @title="Next Page Button"]',
                    )
                )
            )
            driver.execute_script(
                "arguments[0].click();", button
            )  # Ensure no overlay blocks the click
            time.sleep(3)  # Wait for the next page to load
        except Exception as e:
            print(f"Pagination error: {e}")
            break

    # Append the season's data to the main DataFrame
    data = pd.concat([data, season_data], ignore_index=True)

Pagination error: Message: 

Pagination error: Message: 

Pagination error: Message: 

Pagination error: Message: 

Pagination error: Message: 



In [4]:
# Save to CSV
path = "../../Data_NBA_1/nba_stats.csv"
data.to_csv(path, index=False, header=True)
print(f"Data saved to {path}")

# Close the browser
driver.quit()

Data saved to ../../Data_NBA_1/nba_stats.csv
